# TypeScript コーディング問題解答

## 問題文
有効期限付きキャッシュクラスの実装。各キーに有効期限を設定し、期限切れのキーは自動的にアクセス不可になる。

## 1. 問題の分析

### 競技プログラミング視点での分析
- **実行速度最優先**: Map構造でO(1)アクセス、タイムアウトIDの直接管理
- **メモリ使用量最小化**: 期限切れエントリの即座削除、不要なオブジェクト生成回避
- **最適化ポイント**: タイマー管理の効率化、重複処理の排除

### 業務開発視点での分析
- **型安全性**: 厳格な型定義でコンパイル時エラー防止
- **保守性**: 明確な責務分離、分かりやすいメソッド名
- **エラーハンドリング**: 入力値の検証、境界値の適切な処理
- **メモリリーク防止**: タイマーの適切なクリーンアップ

### TypeScript特有の考慮点
- **型推論の活用**: Map<number, CacheEntry>で型安全性を確保
- **readonly修飾子**: 不変性の保証
- **インターフェース定義**: 内部構造の明確化
- **null安全性**: undefined/nullチェックの適切な実施

## 2. アルゴリズムアプローチ比較

| アプローチ | 時間計算量 | 空間計算量 | TS実装コスト | 型安全性 | 可読性 | 備考 |
|---------|----------|----------|------------|---------|--------|------|
| Map + setTimeout | O(1) | O(n) | 低 | 高 | 高 | 最適解 |
| 配列 + 線形探索 | O(n) | O(n) | 低 | 高 | 中 | get/setが遅い |
| 都度チェック方式 | O(1) | O(n) | 中 | 中 | 中 | タイマー不要だがgetで毎回チェック |

## 3. 選択したアルゴリズムと理由

### 選択したアプローチ
**Map + setTimeout方式**

### 理由
- **計算量的な優位性**: すべての操作がO(1)で実行可能
- **TypeScript環境での型安全性**: Map型により強力な型推論が効く
- **保守性・可読性の観点**: Mapの標準メソッドで意図が明確

### TypeScript特有の最適化ポイント
- **型推論**: Mapのジェネリクス型で完全な型安全性
- **インターフェース**: CacheEntry型で構造を明確化
- **strictモード**: コンパイル時にnull/undefinedを厳密チェック
- **NodeJS.Timeout型**: タイマーIDの型安全な管理

## 4. 実装コード

```typescript
// Analyze Complexity
// Runtime 54 ms
// Beats 30.18%
// Memory 54.80 MB
// Beats 74.74%
/**
 * キャッシュエントリの内部構造
 */
interface CacheEntry {
    readonly value: number;
    readonly timeoutId: NodeJS.Timeout;
}

/**
 * 有効期限付きキャッシュクラス
 * @description キーごとに有効期限を設定し、期限切れで自動削除されるキャッシュ
 * @complexity 
 * - set: Time O(1), Space O(1)
 * - get: Time O(1), Space O(1)
 * - count: Time O(1), Space O(1)
 */
class TimeLimitedCache {
    private cache: Map<number, CacheEntry>;
    
    constructor() {
        this.cache = new Map<number, CacheEntry>();
    }
    
    /**
     * キーと値を設定し、有効期限を指定する
     * @param key - キー (0 <= key <= 10^9)
     * @param value - 値 (0 <= value <= 10^9)
     * @param duration - 有効期限（ミリ秒、0 <= duration <= 1000）
     * @returns 既存の未期限切れキーが存在した場合true、それ以外false
     * @complexity Time: O(1), Space: O(1)
     */
    set(key: number, value: number, duration: number): boolean {
        // 型安全性: TypeScriptの型システムで保証されているが、
        // 実行時の追加検証は省略（LeetCode環境では制約条件が保証される）
        
        const existingEntry = this.cache.get(key);
        let hadKey = false;
        
        // 既存エントリが存在する場合、タイマーをクリア
        if (existingEntry !== undefined) {
            clearTimeout(existingEntry.timeoutId);
            hadKey = true;
        }
        
        // 新しいタイマーを設定
        const timeoutId = setTimeout(() => {
            this.cache.delete(key);
        }, duration);
        
        // 新しいエントリを保存
        this.cache.set(key, {
            value,
            timeoutId
        });
        
        return hadKey;
    }
    
    /**
     * キーに対応する値を取得
     * @param key - 取得するキー
     * @returns 未期限切れのキーが存在すれば対応する値、存在しなければ-1
     * @complexity Time: O(1), Space: O(1)
     */
    get(key: number): number {
        const entry = this.cache.get(key);
        
        // undefinedチェック（型安全）
        if (entry === undefined) {
            return -1;
        }
        
        return entry.value;
    }
    
    /**
     * 未期限切れキーの数を取得
     * @returns アクティブなキーの数
     * @complexity Time: O(1), Space: O(1)
     */
    count(): number {
        return this.cache.size;
    }
}

/**
 * const timeLimitedCache = new TimeLimitedCache()
 * timeLimitedCache.set(1, 42, 1000); // false
 * timeLimitedCache.get(1) // 42
 * timeLimitedCache.count() // 1
 */
```

## 5. TypeScript固有の最適化観点

### 型安全性の活用

1. **コンパイル時エラー防止**
   - `Map<number, CacheEntry>`で型が完全に推論される
   - `entry?.value`のようなオプショナルチェーンも可能だが、明示的な`undefined`チェックで意図を明確化

2. **インターフェース定義**
   - `CacheEntry`インターフェースで内部構造を型レベルで文書化
   - `readonly`修飾子でイミュータビリティを表現

3. **null/undefined安全性**
   - `Map.get()`の戻り値が`CacheEntry | undefined`と正確に型付け
   - strictNullChecksにより安全なチェック強制

### 実装上の工夫

1. **メモリリーク防止**
   - `clearTimeout()`で既存タイマーを確実にクリア
   - `setTimeout`のコールバックで自動削除

2. **パフォーマンス最適化**
   - Mapの特性を活かしたO(1)操作
   - 不要なオブジェクト生成を回避

3. **可読性とメンテナンス性**
   - メソッド名が意図を明確に表現
   - JSDocコメントで使用方法を文書化
   - 計算量をコメントで明示

# パフォーマンス改善版の実装

現在の実装は **Runtime 54ms (30.18%)** ということで、改善の余地があります。主な最適化ポイントを分析して改善版を提示します。

## 改善ポイントの分析

### 現在の実装の問題点

1. **setTimeout/clearTimeoutのオーバーヘッド**
   - 各`set`呼び出しで必ずタイマー操作が発生
   - 既存エントリの`clearTimeout`→新規`setTimeout`の2回の呼び出し

2. **Map操作の回数**
   - `get` → `delete` → `set`と複数回のMap操作

3. **オブジェクト生成コスト**
   - 毎回新しい`CacheEntry`オブジェクトを生成

### 最適化戦略

1. **タイマー処理の最適化**: 期限時刻を保存し、遅延チェック方式に変更
2. **Map操作の削減**: 単一の更新操作にまとめる
3. **オブジェクト再利用**: 構造を単純化

## 改善版実装

```typescript
// Analyze Complexity
// Runtime 49 ms
// Beats 55.44%
// Memory 55.56 MB
// Beats 27.72%
/**
 * キャッシュエントリの内部構造（最適化版）
 * - タイマーIDの代わりに期限時刻を保存
 * - よりシンプルな構造で高速化
 */
interface CacheEntry {
    value: number;
    expiresAt: number;  // 期限時刻（ミリ秒）
}

/**
 * 有効期限付きキャッシュクラス（最適化版）
 * @description 遅延削除方式で高速化を実現
 * @complexity 
 * - set: Time O(1), Space O(1) - タイマー操作なし
 * - get: Time O(1), Space O(1) - 期限チェックのみ
 * - count: Time O(n), Space O(1) - 期限切れを除外してカウント
 */
class TimeLimitedCache {
    private cache: Map<number, CacheEntry>;
    
    constructor() {
        this.cache = new Map<number, CacheEntry>();
    }
    
    /**
     * キーと値を設定し、有効期限を指定する
     * @param key - キー
     * @param value - 値
     * @param duration - 有効期限（ミリ秒）
     * @returns 既存の未期限切れキーが存在した場合true、それ以外false
     * @complexity Time: O(1), Space: O(1)
     */
    set(key: number, value: number, duration: number): boolean {
        const now = Date.now();
        const expiresAt = now + duration;
        
        const existingEntry = this.cache.get(key);
        
        // 既存エントリが存在し、かつ未期限切れかチェック
        const hadUnexpiredKey = existingEntry !== undefined && existingEntry.expiresAt > now;
        
        // 新しいエントリを設定（直接更新、タイマー不要）
        this.cache.set(key, { value, expiresAt });
        
        return hadUnexpiredKey;
    }
    
    /**
     * キーに対応する値を取得
     * @param key - 取得するキー
     * @returns 未期限切れのキーが存在すれば対応する値、存在しなければ-1
     * @complexity Time: O(1), Space: O(1)
     */
    get(key: number): number {
        const entry = this.cache.get(key);
        
        if (entry === undefined) {
            return -1;
        }
        
        // 期限切れチェック
        if (entry.expiresAt <= Date.now()) {
            this.cache.delete(key);  // 遅延削除
            return -1;
        }
        
        return entry.value;
    }
    
    /**
     * 未期限切れキーの数を取得
     * @returns アクティブなキーの数
     * @complexity Time: O(n), Space: O(n)
     */
    count(): number {
        const now = Date.now();
        let count = 0;
        
        // 期限切れでないエントリのみカウント
        for (const entry of this.cache.values()) {
            if (entry.expiresAt > now) {
                count++;
            }
        }
        
        return count;
    }
}
```

## さらなる最適化版（積極的クリーンアップ付き）

`count()`の呼び出しが多い場合はこちらの方が高速です：

```typescript
// Analyze Complexity
// Runtime 49 ms
// Beats 55.44%
// Memory 55.31 MB
// Beats 43.33%
/**
 * 有効期限付きキャッシュクラス（積極的クリーンアップ版）
 */
class TimeLimitedCache {
    private cache: Map<number, CacheEntry>;
    
    constructor() {
        this.cache = new Map<number, CacheEntry>();
    }
    
    /**
     * 期限切れエントリを削除（内部ヘルパー）
     * @complexity Time: O(n), Space: O(n)
     */
    private cleanup(): void {
        const now = Date.now();
        
        for (const [key, entry] of this.cache) {
            if (entry.expiresAt <= now) {
                this.cache.delete(key);
            }
        }
    }
    
    set(key: number, value: number, duration: number): boolean {
        const now = Date.now();
        const expiresAt = now + duration;
        
        const existingEntry = this.cache.get(key);
        const hadUnexpiredKey = existingEntry !== undefined && existingEntry.expiresAt > now;
        
        this.cache.set(key, { value, expiresAt });
        
        return hadUnexpiredKey;
    }
    
    get(key: number): number {
        const entry = this.cache.get(key);
        
        if (entry === undefined) {
            return -1;
        }
        
        if (entry.expiresAt <= Date.now()) {
            this.cache.delete(key);
            return -1;
        }
        
        return entry.value;
    }
    
    count(): number {
        this.cleanup();  // count時に一括クリーンアップ
        return this.cache.size;
    }
}
```

## パフォーマンス比較

| 実装方式 | set | get | count | メモリ | 備考 |
|---------|-----|-----|-------|--------|------|
| 元実装（setTimeout） | O(1) + タイマー | O(1) | O(1) | 良 | タイマーオーバーヘッド大 |
| 遅延削除版 | O(1) | O(1) | O(n) | 良 | タイマー不要で高速 |
| 積極削除版 | O(1) | O(1) | O(n) | 最良 | count時に自動クリーンアップ |

## 推奨実装

**遅延削除版（最初の改善版）** を推奨します。理由：

1. **Runtime改善**: setTimeout/clearTimeoutの完全排除で30-40%高速化が期待できる
2. **シンプル**: 実装が明快で保守性が高い
3. **メモリ効率**: 既に74.74%と良好なメモリ使用量を維持
4. **実用的**: LeetCodeの制約条件（最大100アクション）では`count`のO(n)は問題なし

この実装で **Runtime 40ms以下、上位50%以上** を狙えるはずです！